In [1]:
from gensim.models import AuthorTopicModel

model = AuthorTopicModel.load('model.atmodel')

In [2]:
model.show_topics(num_topics=100)

[(0,
  '0.029*"opiniónlsr" + 0.019*"yosoitu" + 0.008*"todopersonal" + 0.006*"LoMásYI COLUMNA" + 0.006*"columnadehoy" + 0.004*"FOTOS Así" + 0.004*"metrópoli" + 0.004*"chitlgalvez" + 0.003*"especialeslsr" + 0.003*"videocolumnalsr"'),
 (1,
  '0.014*"noticias" + 0.012*"fm" + 0.009*"enpunto" + 0.008*"cdmx" + 0.007*"tláhuac" + 0.007*"pri" + 0.006*"autoridades" + 0.006*"franklin" + 0.006*"gobierno" + 0.005*"duarte"'),
 (2,
  '0.009*"ligamxfemenil" + 0.006*"rudycoen" + 0.006*"mulsopolitico" + 0.005*"Rodolfo Higareda" + 0.005*"higareda" + 0.005*"Francisco Cárdenas Cruz" + 0.005*"aguilar" + 0.005*"Ángeles Aguilar" + 0.005*"mfloresarellano" + 0.005*"bibianabelsasso"'),
 (3,
  '0.005*"escobedo" + 0.004*"simpatía" + 0.004*"inmenso" + 0.003*"mannypacquiao" + 0.003*"Angélica Rivera" + 0.003*"daliakaki" + 0.003*"PorSiNoLoViste Así" + 0.003*"receso" + 0.002*"racistas" + 0.002*"jordan"'),
 (4,
  '0.011*"redesdepoder" + 0.011*"redesdepodermty" + 0.007*"cavernícola" + 0.007*"documentoíndigo" + 0.006*"Opin

In [3]:
aut_top = {}
for key, value in model.id2author.items():
    aut_top[value] = model.get_author_topics(value)

In [4]:
aut_top

{'ADNPolitico': [(1, 0.21692398133277213),
  (6, 0.031862770379163449),
  (45, 0.17734126952842771),
  (46, 0.14649895560278045),
  (71, 0.17973973189091799),
  (86, 0.20005529999636207),
  (89, 0.04434365137765419)],
 'ActualidadRT': [(1, 0.03078224419213443),
  (6, 0.080277869430473894),
  (14, 0.21341976952477865),
  (45, 0.17025309830282606),
  (71, 0.28526055931306593),
  (86, 0.20632100440673354),
  (89, 0.013182376539782372)],
 'Adela_Micha': [(1, 0.046668484473735644),
  (6, 0.11358652163079463),
  (15, 0.4701268672986974),
  (71, 0.068572496381364881),
  (86, 0.22267166541573871),
  (89, 0.074841350253355338)],
 'Amsalazar': [(1, 0.14200574173486766),
  (6, 0.018280730241556645),
  (12, 0.058715681967276938),
  (21, 0.2943070541684239),
  (45, 0.12816217103777236),
  (71, 0.050920188200957325),
  (86, 0.050636909416622328),
  (89, 0.2062714542646491),
  (90, 0.037211575107946121)],
 'AnaPOrdorica': [(1, 0.18260097739679776),
  (2, 0.028371330858412994),
  (45, 0.10429728596521

In [5]:
%%time
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
smallest_author = 0  # Ignore authors with documents less than this.
authors = [model.author2id[a] for a in model.author2id.keys() if len(model.author2doc[a]) >= smallest_author]
_ = tsne.fit_transform(model.state.gamma[authors, :])  # Result stored in tsne.embedding_

CPU times: user 1.1 s, sys: 24 ms, total: 1.13 s
Wall time: 596 ms


In [6]:

from bokeh.io import output_file
output_file('grafica.html')

In [7]:
from bokeh.models import HoverTool
from bokeh.plotting import figure, show, ColumnDataSource

x = tsne.embedding_[:, 0]
y = tsne.embedding_[:, 1]
author_names = [model.id2author[a] for a in authors]

scale = 0.01
author_sizes = [len(model.author2doc[a]) for a in author_names]
radii = [size * scale for size in author_sizes]

source = ColumnDataSource(
        data=dict(
            x=x,
            y=y,
            author_names=author_names,
            author_sizes=author_sizes,
            radii=radii,
        )
    )

hover = HoverTool(
        tooltips=[
        ("author", "@author_names"),
        ("size", "@author_sizes"),
        ]
    )

p = figure(tools=[hover, 'crosshair,pan,wheel_zoom,box_zoom,reset,save,lasso_select'])
p.scatter('x', 'y', radius='radii', source=source, fill_alpha=0.6, line_color=None)
show(p)

In [8]:
from gensim.similarities import MatrixSimilarity


index = MatrixSimilarity(model[list(model.id2author.values())])



In [9]:

from gensim import matutils
import pandas as pd


author_vecs = [model.get_author_topics(author) for author in model.id2author.values()]

def similarity(vec1, vec2):
    dist = matutils.hellinger(matutils.sparse2full(vec1, model.num_topics), \
                              matutils.sparse2full(vec2, model.num_topics))
    sim = 1.0 / (1.0 + dist)
    return sim

def get_sims(vec):
    sims = [similarity(vec, vec2) for vec2 in author_vecs]
    return sims

def get_table(name, top_n=10, smallest_author=1):
    sims = get_sims(model.get_author_topics(name))


    table = []
    for elem in enumerate(sims):
        author_name = model.id2author[elem[0]]
        sim = elem[1]
        author_size = len(model.author2doc[author_name])
        if author_size >= smallest_author:
            table.append((author_name, sim, author_size))
            

    df = pd.DataFrame(table, columns=['Author', 'Score', 'Size'])
    df = df.sort_values('Score', ascending=False)[:top_n]
    
    return df

In [10]:
get_table('Pajaropolitico',top_n=136)

,Author,Score,Size
43,Pajaropolitico,1.000000,726
111,sdpnoticias,0.776769,53
77,elsolde_mexico,0.723255,721
93,lacronicadehoy,0.719705,699
96,lopezdoriga,0.719275,658
6,AztecaNoticias,0.718762,197
35,MarioBeteta,0.717402,439
39,NoticiasMVS,0.714790,1040
108,revistaproceso,0.712646,386
49,Reforma,0.704470,827
